In [8]:
from google.colab import files
uploaded = files.upload()


Saving 5000HOUSED.csv to 5000HOUSED (1).csv


In [9]:
import pandas as pd
import io
data1 = pd.read_csv(io.BytesIO(uploaded['5000HOUSED.csv']))
# Dataset is now stored in a Pandas Dataframe

In [3]:
!pip install --pre pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 487 kB 32.8 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 72 kB 810 kB/s 
     |████████████████████████████████| 11.2 MB 66.7 MB/s 
     |████████████████████████████████| 137 kB 68.5 MB/s 
     |████████████████████████████████| 7.0 MB 64.4 MB/s 
     |████████████████████████████████| 3.3 MB 61.5 MB/s 
     |████████████████████████████████| 79.9 MB 1.2 MB/s 
     |████████████████████████████████| 1.4 MB 48.6 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 2.0 MB 37.9 MB/s 
     |████████████████████████████████| 281 kB 70.3 MB/s 
     |████████████████████████████████| 106 kB 62.7 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Prep

In [10]:
from pycaret.datasets import get_data
data1 
data = data1.dropna()
data


,date,totalusage
0,9/04/2015,0.002190
1,9/04/2015,0.048108
2,9/04/2015,0.060040
3,9/04/2015,0.052533
4,9/04/2015,0.022288
...,...,...
4994,11/12/2015,0.021016
4995,11/12/2015,0.020588
4996,11/12/2015,0.038688
4997,11/12/2015,0.020894


In [11]:
!pip uninstall matplotlib
!pip install matplotlib==3.1.3
import matplotlib.pyplot as plt


Found existing installation: matplotlib 3.1.3
Uninstalling matplotlib-3.1.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/matplotlib-3.1.3-py3.7-nspkg.pth
    /usr/local/lib/python3.7/dist-packages/matplotlib-3.1.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/matplotlib/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axes_grid/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axes_grid1/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axisartist/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/mplot3d/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/tests/*
    /usr/local/lib/python3.7/dist-packages/pylab.py
Proceed (y/n)? Y
  Successfully uninstalled matplotlib-3.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.1.3-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the pa

In [12]:
data.dtypes
data['date'] = pd.to_datetime(data['date'])
# create 30 day  moving average
data['MA30'] = data['totalusage'].rolling(30).mean()

# plot the data and MA
import plotly.express as px
fig = px.line(data, x="date", y=["totalusage", "MA30"], template = 'plotly_dark')
fig.show()

In [13]:

## extract month and year from dates
import numpy as np

# extract features from date**
data['month'] = [i.month for i in data['date']]
data['year'] = [i.year for i in data['date']]
# data['day_of_week'] = [i.dayofweek for i in data['date']]
data['day'] = [i.day for i in data['date']]
# data['day_of_year'] = [i.dayofyear for i in data['date']]


data['Series'] = np.arange(1,len(data)+1)
#drop unnecessary columns and re-arrange
data.drop([ 'MA30'], axis=1, inplace=True)
# check the head of the dataset

data = data[['Series','date','totalusage','day','month','year']]
# 'day_of_year''day_of_week',
data.head()

,Series,date,totalusage,day,month,year
0,1,2015-09-04,0.002190,4,9,2015
1,2,2015-09-04,0.048108,4,9,2015
2,3,2015-09-04,0.060040,4,9,2015
3,4,2015-09-04,0.052533,4,9,2015
4,5,2015-09-04,0.022288,4,9,2015


In [14]:
print(data.isnull().sum())
data['totalusage'].round(decimals = 5)
data.info()

Series        0
date          0
totalusage    0
day           0
month         0
year          0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4999 entries, 0 to 4998
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Series      4999 non-null   int64         
 1   date        4999 non-null   datetime64[ns]
 2   totalusage  4999 non-null   float64       
 3   day         4999 non-null   int64         
 4   month       4999 non-null   int64         
 5   year        4999 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 273.4 KB


In [15]:
# import numpy as np
# from sklearn.model_selection import train_test_split

# import datetime 
# split_date = datetime.date(2015,12,13)
# train = data[(pd.to_datetime(data["date"]).dt.date < split_date)]
# test = data[(pd.to_datetime(data["date"]).dt.date > split_date)]
# train.shape, test.shape

In [16]:
# import the regression module
from pycaret.regression import *
# initialize setup
s = setup( data, target = 'totalusage', fold_strategy = 'timeseries', fold = 3, session_id = 123)



,Description,Value
0,Session id,123
1,Target,totalusage
2,Target type,Regression
3,Data shape,"(4999, 6)"
4,Train data shape,"(3499, 6)"
5,Test data shape,"(1500, 6)"
6,Numeric features,4
7,Date features,1
8,Preprocess,True
9,Imputation type,simple


In [17]:
best = compare_models(sort = 'MAE',errors="raise")


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.0157,0.0060,0.0761,0.2454,0.0533,0.2670,0.4967
rf,Random Forest Regressor,0.0168,0.0064,0.0786,0.1893,0.0548,0.3001,0.3633
dt,Decision Tree Regressor,0.0169,0.0088,0.0928,-0.1551,0.0622,0.2782,0.0400
knn,K Neighbors Regressor,0.0204,0.0065,0.0790,0.1929,0.0563,0.4169,0.1067
huber,Huber Regressor,0.0248,0.0087,0.0913,-0.0776,0.0680,0.2061,0.0500
lightgbm,Light Gradient Boosting Machine,0.0263,0.0070,0.0822,0.1301,0.0594,0.6683,0.2633
gbr,Gradient Boosting Regressor,0.0263,0.0072,0.0844,0.0455,0.0593,0.6343,0.4000
lr,Linear Regression,0.0384,0.0081,0.0878,0.0081,0.0646,1.1219,1.8700
br,Bayesian Ridge,0.0386,0.0081,0.0878,0.0081,0.0646,1.1280,0.0367
ridge,Ridge Regression,0.0389,0.0081,0.0880,0.0048,0.0647,1.1382,0.0333


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

In [18]:
prediction_holdout = predict_model(best);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.0190,0.0120,0.1094,0.2800,0.0673,0.2297


In [19]:
# generate predictions on the original dataset
predictions = predict_model(best, data=data)
# line plot

predictions['Date']= data['date']

fig = px.line(predictions, x='Date', y=["totalusage", "prediction_label"], template = 'plotly_dark')
# add a vertical rectange for test-set separation
# fig.add_vrect(x0="-01-01", x1="1960-12-01", fillcolor="grey", opacity=0.25, line_width=0)
fig.show()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.0057,0.0036,0.0599,0.6634,0.0369,0.0689


In [20]:
predictions

,Series,date_month,date_day,day,month,totalusage,prediction_label,Date
0,1.0,9.0,4.0,4.0,9.0,0.002190,0.002190,2015-09-04
1,2.0,9.0,4.0,4.0,9.0,0.048108,0.048108,2015-09-04
2,3.0,9.0,4.0,4.0,9.0,0.060040,0.060040,2015-09-04
3,4.0,9.0,4.0,4.0,9.0,0.052533,0.052533,2015-09-04
4,5.0,9.0,4.0,4.0,9.0,0.022288,0.040065,2015-09-04
...,...,...,...,...,...,...,...,...
4994,4995.0,11.0,12.0,12.0,11.0,0.021016,0.020679,2015-11-12
4995,4996.0,11.0,12.0,12.0,11.0,0.020588,0.020588,2015-11-12
4996,4997.0,11.0,12.0,12.0,11.0,0.038688,0.038688,2015-11-12
4997,4998.0,11.0,12.0,12.0,11.0,0.020894,0.020894,2015-11-12


In [21]:
final_best = finalize_model(best)

In [22]:
future_dates = pd.date_range(start = '12/11/2015', end = '12/02/2016', freq = 'D')
future_df = pd.DataFrame()
future_df['date'] = pd.date_range(start = '12/11/2015', end = '12/02/2016', freq = 'D')
future_df['month'] = [i.month for i in future_dates]
future_df['year'] = [i.year for i in future_dates]    
future_df['day'] = [i.day for i in future_dates]
future_df['Series'] = np.arange(4999,(4999+len(future_dates)))
future_df

,date,month,year,day,Series
0,2015-12-11,12,2015,11,4999
1,2015-12-12,12,2015,12,5000
2,2015-12-13,12,2015,13,5001
3,2015-12-14,12,2015,14,5002
4,2015-12-15,12,2015,15,5003
...,...,...,...,...,...
353,2016-11-28,11,2016,28,5352
354,2016-11-29,11,2016,29,5353
355,2016-11-30,11,2016,30,5354
356,2016-12-01,12,2016,1,5355


In [23]:
predictions_future = predict_model(final_best, data=future_df)
predictions_future

,Series,date_month,date_day,day,month,prediction_label
0,4999.0,12.0,11.0,11.0,12.0,0.021157
1,5000.0,12.0,12.0,12.0,12.0,0.020918
2,5001.0,12.0,13.0,13.0,12.0,0.020905
3,5002.0,12.0,14.0,14.0,12.0,0.020898
4,5003.0,12.0,15.0,15.0,12.0,0.021682
...,...,...,...,...,...,...
353,5352.0,11.0,28.0,28.0,11.0,0.023285
354,5353.0,11.0,29.0,29.0,11.0,0.023253
355,5354.0,11.0,30.0,30.0,11.0,0.028484
356,5355.0,12.0,1.0,1.0,12.0,0.382119


In [25]:
data = data[['Series','date','totalusage','day','month','year']]

predictions_future.rename(columns = {'prediction_label':'totalusage'}, inplace = True)
# predictions_future.drop([ 'date_month'])
# predictions_future.drop([ 'date_day'])
predictions_future['year'] = future_df['year']
predictions_future['date'] = future_df['date']

predictions_future = predictions_future[['Series','date','totalusage','day','month','year']]
predictions_future

,Series,date,totalusage,day,month,year
0,4999.0,2015-12-11,0.021157,11.0,12.0,2015
1,5000.0,2015-12-12,0.020918,12.0,12.0,2015
2,5001.0,2015-12-13,0.020905,13.0,12.0,2015
3,5002.0,2015-12-14,0.020898,14.0,12.0,2015
4,5003.0,2015-12-15,0.021682,15.0,12.0,2015
...,...,...,...,...,...,...
353,5352.0,2016-11-28,0.023285,28.0,11.0,2016
354,5353.0,2016-11-29,0.023253,29.0,11.0,2016
355,5354.0,2016-11-30,0.028484,30.0,11.0,2016
356,5355.0,2016-12-01,0.382119,1.0,12.0,2016


In [26]:
concat_df = pd.concat([data,predictions_future], axis=0)
concat_df_i = pd.date_range(start='01/01/2014',  end = '15/06/2015', freq = 'D')
fig = px.line(concat_df, x="date", y=["totalusage"], template = 'plotly_dark')
fig.show()